# Preparation

In [ ]:
import os
import random
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import torch

# Get title and URLS

In [ ]:
jans = ["domestic", "world", "business", "entertainment", "sports", "it", "science", "local"]

In [ ]:
def make_URL(idx: int, jan: str) -> str:
  return f'https://news.yahoo.co.jp/topics/{jan}?page={idx}'

URLexample = make_URL(1, "domestic")
print(URLexample)

https://news.yahoo.co.jp/topics/domestic?page=1


In [ ]:
URL_ = URLexample
rest = requests.get(URL_)

soup = BeautifulSoup(rest.text, "html.parser")

data_list = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))

In [ ]:
soup

<!DOCTYPE html>

<html>
<head>
<title>ããç¥ãããæ¬§å·çµæ¸é åï¼EEAï¼ããã³ã¤ã®ãªã¹ãããå©ç¨ã®ãå®¢æ§ã¸ - Yahoo! JAPAN</title>
<meta charset="utf-8"/>
<meta content="width=device-width,user-scalable=no, viewport-fit=cover" name="viewport"/>
<!-- riff -->
<link href="https://s.yimg.jp/images/riff/4.7.1/riff.min.css" rel="stylesheet"/>
<!-- favicon -->
<link href="https://s.yimg.jp/c/icon/s/bsc/2.0/favicon.ico" rel="icon"/>
<script type="text/javascript">
    window.onload = onLoad;
    function onLoad() {
      YJ_IS_SORRYPAGE = true;
    }
  </script>
</head>
<body>
<div id="wrapper">
<!-- ãããã¼ -->
<header>
<!-- basic-MHCSS -->
<!-- mhcss -->
<link href="https://s.yimg.jp/yui/jp/mh/pc/1.5.6/css/std.css" media="all" rel="stylesheet" type="text/css"/>
<!-- /mhcss -->
<!-- basic-MH-login -->
<!-- mhd -->
<div id="msthd">
<div>
<div id="msthdtp">
<div id="mhd_uhd_pc"></div>
<div id="msthdsb">
<div class="msthdLink">
<ul>
<!-- mhhl -->
<li class="msthd

In [ ]:
def get_title_url(URL: str, concat: dict) -> [dict, bool]:
  URL_ = URL
  rest = requests.get(URL_)

  soup = BeautifulSoup(rest.text, "html.parser")

  data_list = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))
  if len(data_list) == 0:
    continue_ = False
  else:
    for data in data_list:
      if data.div:
        concat[data.find_all('div')[3].string] = [data.attrs["href"], data.find_all('div')[4].string]
      continue_ = True
  return concat, continue_

concat = {}
concat, continue_ = get_title_url(URLexample, concat)
print(concat)
print(continue_)

{}
False


In [ ]:
def  make_jan_title_url(jan: str) -> dict:
  continue_ = True
  idx = 1
  concat = {}
  while continue_ != False:
    URL = make_URL(idx, jan)
    concat, continue_ = get_title_url(URL, concat)
    idx += 1
  return concat

concat = make_jan_title_url('domestic')
print(concat)

{}


# Loop for getting title and URL

In [ ]:
concat_jan = {}
for jan in jans:
  concat = make_jan_title_url(jan)
  concat_jan[jan] = concat

In [ ]:
concat_jan

{'domestic': {},
 'world': {},
 'business': {},
 'entertainment': {},
 'sports': {},
 'it': {},
 'science': {},
 'local': {}}

In [ ]:
URLexample = list(concat_jan['domestic'].values())[5]

IndexError: ignored

In [ ]:
URLexample

# Extract document

In [ ]:
def get_document(URL: str) -> [str, int]:
  html_doc = requests.get(URL).text
  soup = BeautifulSoup(html_doc, 'html.parser')
  article_list = soup.find_all(href=re.compile("news.yahoo.co.jp/"))
  for idx, article in enumerate(article_list):
    if "記事全文を読む" in str(article):
      article_url = article.attrs['href']
      break
    else:
      article_url = None
      if idx == len(article_list) - 1:
        return np.nan

  html_doc = requests.get(article_url).text
  soup = BeautifulSoup(html_doc, 'html.parser')

  try:
      comment_count = int(str(soup.find_all('div', {'class': "riff-VisuallyHidden__root"})).split('<!-- -->')[1])
  except IndexError:
      comment_count = 'NaN'
  
  ps = soup.find_all("p", {"class": "sc-jtggT bAhgUU yjSlinkDirectlink highLightSearchTarget"})
  
  texts = []
  for p in ps:
    texts.append(p.get_text())
  texts = [sentence.strip() for sentence in texts]  # 空白文字(スペースやタブ、改行)の削除
  texts = list(filter(lambda line: line != '', texts))
  texts = ''.join(texts)
  texts = texts.translate(str.maketrans(
          {'\n': '', '\t': '', '\r': '', '\u3000': '', ' ': ''}))  # 改行やタブ、全角スペースを消す
  return [texts, comment_count]

output = get_document(URLexample)
print(output[0])
print(output[1])

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for jan in tqdm(jans):
  titles = list(concat_jan[jan].keys())
  urls = [i[0] for i in list(concat_jan[jan].values())]
  dates = [i[1] for i in list(concat_jan[jan].values())]
  texts = []
  comment_counts = []
  for url in tqdm(urls):
    texts.append(get_document(url)[0])
    comment_counts.append(get_document(url)[1])
  
  if jan == 'domestic':
    df = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count'])
    df['title'] = titles
    df['document'] = texts
    df['comment_count'] = comment_counts
    df['date'] = dates
    df['genre'] = jan
  else:
    df_jan = pd.DataFrame(columns=['genre', 'title', 'date', 'document'])
    df_jan['title'] = titles
    df_jan['document'] = texts
    df_jan['comment_count'] = comment_counts
    df_jan['date'] = dates
    df_jan['genre'] = jan
    df = pd.concat([df, df_jan])

In [ ]:
df = pd.read_csv('/content/result_yahoo_totyu.csv')

In [ ]:
df

,genre,title,date,document,comment_count
0,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0
1,domestic,絞首刑は残虐 死刑囚3人が国提訴,11/29(火) 11:55,絞首刑による死刑執行は残虐な刑罰を禁じる憲法などに違反するとして、確定死刑囚3人が29日、国...,8239.0
2,domestic,五輪談合 ADK側にも家宅捜索,11/29(火) 11:26,東京オリンピック・パラリンピックのテスト大会事業をめぐる談合事件で、東京地検特捜部などは29...,618.0
3,domestic,介護保険の2割負担 対象拡大へ,11/29(火) 8:01,厚生労働省は２８日、介護保険サービス利用時の自己負担割合が２割となる対象者を拡大する方針を社...,1238.0
4,domestic,五輪組織委 受注候補の一覧作成か,11/29(火) 7:31,東京五輪・パラリンピックのテスト大会業務をめぐる入札談合事件で、大会組織委員会が入札前に、受...,869.0
...,...,...,...,...,...
554,local,馬に蹴られたか 馬術部生徒が流血,11/23(水) 17:05,青森県十和田市の農業高校で、馬術部の女子生徒がけがをして倒れているのが見つかりました。馬に蹴...,421.0
555,local,前回W杯で若者ら騒ぐ 大阪で警戒,11/23(水) 15:34,ＦＩＦＡワールドカップ（Ｗ杯）カタール大会は２３日午後１０時（日本時間）、日本代表がドイツと...,375.0
556,local,マナー悪すぎトイレ撤去 住職苦渋,11/23(水) 14:38,「残念ながら境内のトイレは撤去することになりました」。兵庫県西宮市、六甲山の麓にある高野山真...,4491.0
557,local,沈没事故から7カ月 今も続く献花,11/23(水) 12:48,北海道知床沖で起きた観光船の沈没事故から7か月となりました。献花台には今も多くの花が手向けら...,94.0


In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [ ]:
from wordcloud import WordCloud
import glob 
import collections
import MeCab
FONT_PATH = "/usr/share/fonts/truetype/ipagp.ttf"
TXT_NAME = "/content/text"
 
 
def get_word_str(text):

    code_regex = re.compile('[\t\s!"#$%&\'\\\\()*+,-./:;；：<=>?@[\\]^_`{|}~○｢｣「」〔〕“”〈〉'\
    '『』【】＆＊（）＄＃＠？！｀＋￥¥％♪…◇→←↓↑｡･ω･｡ﾟ´∀｀ΣДｘ⑥◎©︎♡★☆▽※ゞノ〆εσ＞＜┌┘]')
    text = code_regex.sub('', text)
    #text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    mecab = MeCab.Tagger('mecab-ipadic-neologd')
    parsed = mecab.parse(text)
    lines = parsed.split('\n')
    lines = lines[0:-2]
    word_list = []
 
    for line in lines:
        tmp = re.split('\t|,', line)
 
        # 名詞のみ対象
        if tmp[1] in ["名詞"]:
          word_list.append(tmp[0])
 
    return word_list


def get_count(text: str) -> list:

  # 文字列取得
  word_list = get_word_str(text)

  #stopwordsの設定
  words = ["私","わたし","僕","あなた","みんな","ただ","ほか","それ", "もの", "の", "さん", "横井", "これ", "よう", "ところ","ため","うち","ここ","そう","どこ", "つもり", "いつ","あと","もん","はず","こと","そこ","あれ","なに","傍点","まま","事","人","方","何","時","一","二","三","四","五","六","七","八","九","十", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "１", "２", "３", "４", "５", "６", "７", "８", "９", "１０"]

  for word in words:
    if word in word_list:
      word_list = list(filter((word).__ne__, word_list))

  c = collections.Counter(word_list)
  list_item1 = []
  for i in list(c.items()):
    try:
      int(i[0])
    except ValueError:
      list_item1.append(i)
  
  list_item2 = []
  for i in list_item1:
    if len(i[0]) != 1:
      list_item2.append(i)
  return list_item2

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.reset_index(drop=True)
df_results = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count', 'word', 'count'])
for  idx, i in enumerate(range(df.shape[0])):
    df_result = pd.DataFrame(columns=['genre', 'title', 'date', 'document', 'comment_count', 'word', 'count'])
    df_ = df.iloc[[i]]
    genre = df_['genre'].values[0]
    title = df_['title'].values[0]
    date = df_['date'].values[0]
    text = df_['document'].values[0]
    comment_count = df_['comment_count'].values[0]
    if  text is not np.nan:
      word_list = get_count(text)
      words = [j[0]for j in word_list]
      count = [j[1]for j in word_list]
      df_result['word'] = words
      df_result['count'] = count
      df_result['genre'] = genre
      df_result['title'] = title
      df_result['date'] = date
      df_result['document'] = text
      df_result['comment_count'] = comment_count
    
      df_results = pd.concat([df_results, df_result], axis=0)

In [ ]:
df_results_copy = df_results.copy()
df_results_copy['genre'] = "all"
df_results = pd.concat([df_results, df_results_copy], axis=0)

In [ ]:
df_results

,genre,title,date,document,comment_count,word,count
0,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0,永岡,1
1,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0,桂子,1
2,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0,文部,1
3,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0,科学,1
4,domestic,コロナで共通テスト欠席 救済縮小,11/29(火) 12:42,永岡桂子文部科学相は29日の閣議後記者会見で、来春入学者の大学入試に関し、新型コロナウイルス...,235.0,閣議,1
...,...,...,...,...,...,...,...
26,local,レインボーブリッジ封鎖 催しで,11/23(水) 10:44,レインボーブリッジが封鎖され、自転車が駆け抜けました。東京都の「レインボーライド」は、環境に...,683.0,地区,1
27,local,レインボーブリッジ封鎖 催しで,11/23(水) 10:44,レインボーブリッジが封鎖され、自転車が駆け抜けました。東京都の「レインボーライド」は、環境に...,683.0,周辺,1
28,local,レインボーブリッジ封鎖 催しで,11/23(水) 10:44,レインボーブリッジが封鎖され、自転車が駆け抜けました。東京都の「レインボーライド」は、環境に...,683.0,正午,1
29,local,レインボーブリッジ封鎖 催しで,11/23(水) 10:44,レインボーブリッジが封鎖され、自転車が駆け抜けました。東京都の「レインボーライド」は、環境に...,683.0,交通,1


In [ ]:
df_results.to_csv('result_yahoo.csv', index = None)